 # The flying sidekick traveling salesman problem: Optimization of drone-assisted parcel delivery

# The parallel drone scheduling TSP

## Importa la libreria GRB inizializza il problema

In [ ]:
import sys
import math
import random
from itertools import permutations
import networkx as nx 
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

# Inizializza il modelo
m = gp.Model('The parallel drone scheduling TSP')

## Definizione di una funzione subtour che mi tiene traccia del percorso del truck

In [ ]:
def subtour(vals):
    cycle = []  # Inizializza una lista per contenere il ciclo
    nodes = set(i for i, j in vals.keys() if vals[i, j] > 0.5)  # Ottiene l'insieme dei nodi con valore 1 nella soluzione
    visited_nodes = set()  # Inizializza un insieme per tenere traccia dei nodi visitati

    while nodes:  # Continua finché ci sono nodi non visitati
        current_node = nodes.pop()  # Estrae un nodo da quelli non visitati
        visited_nodes.add(current_node)  # Aggiunge il nodo a quelli visitati
        cycle.append(current_node)  # Aggiunge il nodo al ciclo corrente

        while True:
            next_node_candidates = [j for i, j in vals.keys() if i == current_node and vals[i, j] > 0.5]  # Ottiene i nodi successivi

            if not next_node_candidates or next_node_candidates[0] in visited_nodes:  # Se non ci sono nodi successivi o sono già visitati, interrompi il ciclo
                break

            next_node = next_node_candidates[0]  # Estrae il nodo successivo
            if next_node in nodes:
                nodes.remove(next_node)  # Rimuove il nodo successivo dagli "unvisited"
            
            visited_nodes.add(next_node)  # Aggiunge il nodo successivo a quelli visitati
            cycle.append(next_node)  # Aggiunge il nodo successivo al ciclo corrente
            current_node = next_node  # Imposta il nodo corrente al successivo
            
    return cycle  # Restituisce il ciclo

## Lettura da file dei dati di input

In [ ]:
#LETTURA DA FILE PER NUMERO DI CLIENTI E DRONI

file = open("consegne_tempo_40_clienti.txt", "r") #Apro il file di input in modo tale da leggere tutti i dati di interesse

line = file.readlines() #Leggo le righe del file

c = int(line[0].strip("\n").split(" ")[2]) #Leggo dal file il numero di clienti ai quali bisogna consegnare un pacco
d = int(line[2].strip("\n").split(" ")[2]) #Leggo dal file il numero di UAV da utilizzare per la consegna in aggiunta al truck

N = range(0,c+2) #range di valori per indicare tutti i nodi nella rete
N0 = range(0,c+1) #range di valori per indicare i nodi da cui un veicolo può partire
N1 = range(1,c+2) #range di valori per indicare i nodi che un veicolo può visitare durante il corso di un tour.
C = range(1,c+1) #range di valori per indicare l'insieme di tutti i clienti
V = range(d) #range di valori per indicare l'insieme degli UAV

PosClienti = {} #Posizioni di tutti i clienti
PosNodi = {} #Posizione di tutti i nodi della rete (compresi i due nodi di andata e ritorno che mi indicano il deposito)

#Creo un dizionario che ha come chiave il nodo i-esimo e come valore la coppia di coordinate cartesiane del nodo
for k in range(0,c+2):
    row = line[4+k].strip("\n").split(" ")
    PosNodi[int(row[0])] = (float(row[1]), float(row[2]))

P = [] #Vettore Pesi Pacchi
CL = [] #Vettore Clienti (utilizzato per calcolo LB)

#Come prima creo un dizionario ma solo con i nodi clienti. Inoltre creo due vettori, uno contenente il peso dei pacchi da
#consegnare ai clienti e uno contenente i clienti stessi (ai fini del LB non posso usare C definito come range)
for k in range(c):
    row = line[5+k].strip("\n").split(" ")
    PosClienti[int(row[0])] = (float(row[1]), float(row[2]))
    P.append(float(row[3]))
    CL.append(int(row[0]))

print("Posizione dei clienti ai quali effettuare la consegna:", '\n', PosClienti, '\n')   
print("Posizione di tutti i nodi della rete:", '\n', PosNodi, '\n')   

C_P = [] #C’ è un sottoinsieme di C e rappresenta l'insieme di quei clienti che possono ricevere la consegna
#del loro pacco tramite UAV in quanto il peso del pacco da consegnare non supera la capacità massima di carico dell'UAV
#Creo C’ leggendo da file i clienti che soddisfano la condizione sopra scritta

for k in range(c):
    row = line[5+k].strip("\n").split(" ")
    if float(row[3]) <= 5.0:
        C_P.append(int(row[0]))

print("Clienti appartenenti all'insieme C':", '\n', C_P, '\n')   

file.close; #Chiusura del file

## Definizione delle variabili decisionali e dei costi di ogni arco

In [ ]:
#Calcolo della distanza euclidea tra i vari nodi della rete
dist = {(i, j):
        math.sqrt(sum((PosNodi[i][k]-PosNodi[j][k])**2 for k in range(2)))
        for i in range(0,c+2) for j in range(0,c+2) if i != j}

print("Distanze Euclidee tra i vari nodi della rete:", '\n', dist, '\n')

#Velocità in km/h
vel_truck = 40
vel_UAV = 50

#Calcolo dei tempi tau del truck
tau = {(i, j):
      round(dist[i,j]/vel_truck,3) for i,j in dist.keys()}
#Utilizzo 'round(-,3)' per approssimare il valore ottenuto alle prime 3 cifre decimali

print("Tempi di consegna da un punto ad un altro del truck:", '\n', tau, '\n')

#tau'(0,i)+tau'(i,c+1) <= e
#e costante di tempo. Nel nostro esempio essa è pari a 0.5, ovvero 30 minuti
#tau'(0,i) = tempo che l'UAV impiega per consegnare il pacco dal deposito al cliente i
#tau'(i,c+1) = tempo che l'UAV impiega per tornare al deposito una volta consegnato il pacco al cliente i
tau1 = {}
C_S = [] #C’’ è un sottoinsieme di C’. Esso denota quei clienti, idonei alla consegna tramite UAV,
#che sono entro la portata dell'UAV dal centro di distribuzione (deve essere rispettata la relazione sopra scritta)

#Relazione tra tempo di andata e tempo di ritorno di un UAV che va da un cliente che appartiene all'insieme C''
#T_R=T_A-T_A*(1-3/4)*(P/3)
#P=Peso del Pacco, T_A=Tempo di Andata, T_R=Tempo di Ritorno

#Valuto tra i clienti appartenenti all'insieme C’ quali rispettano la condizione di tempo di consegna del drone e creo
#un vettore C_S che contiene questi ultimi
for i in C_P:
    tau1[0,i]=round(dist[0,i]/vel_UAV,3)
    T_A=tau1[0,i]
    tau1[i,c+1]=round(T_A-T_A*(1-3/4)*P[i-1]/3,3)
    T_R=tau1[i,c+1]
    if T_A+T_R <= 0.5:
        C_S.append(i)  

print("Clienti appartenenti all'insieme C'':", '\n', C_S, '\n') 

#Definisco la variabile x[i,j] che è pari a 1 se il truck si sposta dal nodo i∈ N0 al nodo j∈{N+:j≠i} 
x = m.addVars(tau.keys(), obj=tau, vtype=GRB.BINARY, name = 'x')

#Definisco la variabile y[i,v] che è pari a 1 se il cliente i ∈ C’’ è servito dall'UAV v∈ V.
y = m.addVars(C_S,d, obj=tau1, vtype=GRB.BINARY,name = 'y')

#Definisco la variabile decisionale ausiliaria u che mi permette di eliminare i possibili sottogiri
#u[i] non è altro che l'ordine di visita del vertice i nella soluzione
u = {}
for i in N:
    u[i] = m.addVar(obj=0,vtype=GRB.INTEGER,lb=0,ub=c+2,name='u'+str(i)) #La variabile u deve essere compresa tra 1 e c+2
    if i == 0:
        m.addConstr(u[i]==0) #Il nodo di partenza deve avere un valore di u associato pari a 0 essendo il primo nodo visitato

#print("Variabile x:", '\n', x,'\n')
#print("Variabile y:", '\n', y,'\n')
#print("Variabile u:", '\n', u, '\n')

## Definizione della funzione obiettivo

In [ ]:
#Variabile che andrò a mettere nella funzione obiettivo per linearizzare il problema di min-MAX che è NON LINEARE
#Essendo un problema di min-MAX avrò vari vincoli di >= su questa variabile
z = m.addVar(vtype=GRB.CONTINUOUS, name = 'z')

#Definizione della funzione obiettivo (problema di MINIMIZZAZIONE)
#La funzione obiettivo Cerca di minimizzare l'orario di ritorno più tardivo al deposito sia per l'UAV che per il truck
obj = m.setObjective(z, GRB.MINIMIZE)

## Definizione dei vincoli

In [ ]:
#Limite inferiore su z basato sull'assegnazione del truck
m.addConstr( z >= gp.quicksum(tau[i,j]*x[i,j] for i in N0 for j in N1 if j != i))

#Limite inferiore su z basato sull'assegnazione dell'UAV
for v in V:
    m.addConstr( z >= gp.quicksum((tau1[0,i]+tau1[i,c+1]) * y[i,v] for i in C_S))
    
#Ogni cliente è visitato una sola volta sia dal truck (prima sommatoria) che dall'UAV (seconda sommatoria)   
for j in C:
    m.addConstr((gp.quicksum(x[i,j] for i in N0 if i != j) + gp.quicksum(y[j,v] for v in V if j in C_S)) == 1)

#Il truck deve lasciare il deposito esattamente una volta
m.addConstr(gp.quicksum(x[0,j] for j in N1) == 1)

#Il truck deve ritornare al deposito
m.addConstr(gp.quicksum(x[i,c+1] for i in N0) == 1)

#Quando il truck entra in un nodo cliente deve anche lasciare quel nodo
for j in C:
    m.addConstr(gp.quicksum(x[i,j] for i in N0 if i != j) == gp.quicksum(x[j,k] for k in N1 if k != j))

#Vincolo standard di eliminazione dei sottocircuiti
m.addConstrs(u[i]-u[j]+1 <= (c+2)*(1-x[i,j]) for i in N for j in N1 if j != i);

## Ottimizzazione

In [ ]:
# Ottimizzazione del modello
m.optimize()

m._vars = x
vals = m.getAttr('X', x)
tour = subtour(vals)

# Stampa il percorso ottimale e il costo associato
print('')
print('Tour Ottimo del Truck: %s' % str(tour))
print('Costo Ottimo: %g' % m.ObjVal)
print('')

#Stampa dei clienti raggiunti dagli UAV
for i,v in y.keys():
    if y[i,v].x > 0.5:
        print("L'UAV",str(v+1),"ha raggiunto il cliente",str(i))

## Visualizzazione grafica della posizione dei clienti

In [ ]:
G = nx.DiGraph()

# Aggiunta dei nodi
G.add_nodes_from(N)

# Aggiunta degli archi
G.add_edges_from(tau)
plt.figure(figsize=(25,20))
nx.draw(G, PosNodi, with_labels=True, node_size=2000)
#nx.draw_networkx_edge_labels(G,PosNodi, edge_labels=tau, label_pos=0.75)
plt.draw()

## Visualizzazione grafica della posizione dei clienti visitabili dall'UAV per il vincolo di peso

In [ ]:
G1 = nx.DiGraph()

#Aggiunta dei nodi
G1.add_nodes_from(N)

# Aggiunta degli archi
G1.add_edges_from(tau1)

plt.figure(figsize=(15,10))
nx.draw(G1, PosNodi, with_labels=True, node_size=700)
#nx.draw_networkx_edge_labels(G1,PosNodi, edge_labels=tau1, label_pos=0.75)
plt.draw()

## Percorso ottimo del Truck

In [ ]:
plt.figure(figsize=(18,15))
def plot_tsp_solution(tour, pos, title=""):
    G2 = nx.DiGraph()

    # Aggiunta dei nodi al grafo
    for node in pos:
        G2.add_node(node, pos=pos[node])

    # Aggiunta degli archi al grado
    for i in range(len(tour) - 1):
        G2.add_edge(tour[i], tour[i + 1])

    # Percorso ottimale
    G2.add_edge(tour[-1], tour[0])

    # Posizione dei nodi
    node_positions = nx.get_node_attributes(G2, 'pos')

    # Grafo
    nx.draw(G2, pos=node_positions, with_labels=True, node_size=700, font_size=15, font_color='black', node_color='royalblue', edge_color='white', linewidths=2)

    # Archi del percorso ottimale
    nx.draw_networkx_edges(G2, pos=node_positions, edgelist=[(tour[i], tour[i + 1]) for i in range(len(tour) - 1)], edge_color='skyblue', width=4.0)
   
    # Aggiunta titolo
    plt.title(title)

    # Visualizzazioen grafico
    plt.show()


#Visualizzazione percorso ottimale
plot_tsp_solution(tour, PosNodi, title="Percorso Ottimale Truck")


# Calcolo LB

## LB con metodi pratici

In [ ]:
#Considero solo i clienti non appartenenti all'insieme C’’ in quanto sono quei clienti che possono essere raggiunti 
#solo ed esclusivamente dal Truck in modo tale da ottenere sicuramente un LB non ammissibile
C_NS = [elem for elem in CL if elem not in C_S] #Clienti non appartenenti all'insieme C’’

print('\n', "Clienti non appartenenti all'insieme C’’: ",C_NS)

if len(C_NS) != 0:
    C_NS.insert(0,0) #Aggiungo il nodo deposito (nella matrice svolge sia il ruolo di nodo di partenza che nodo di arrivo(13))
matrice_vertice_arco_1 = [[tau.get((i,j),0) if i != j else 0 for j in C_NS] for i in C_NS] #Costruzione matrice con costi tau 
#Stampa matrice
#for riga in matrice_vertice_arco_1:
#    print(riga)
LB1 = sum([min(filter(lambda x: x!= 0,riga)) for riga in matrice_vertice_arco_1]) #Somma dei minimi di ogni riga
LB1 = round(LB1,3)
print('\n',"LB1: ", LB1)

## LB con rilassamento dei vincoli di interezza

In [ ]:
#Rilasso ora i vincoli di interezza e ottimizzo il nuovo modello
m_r = gp.Model('TSP Truck')
xr = m_r.addVars(tau.keys(), obj=tau, lb=0, ub=1, vtype=GRB.CONTINUOUS, name = 'xr')
yr = m_r.addVars(C_S,d, obj=tau1, vtype=GRB.BINARY,name = 'yr')
ur = {}
for i in N:
    ur[i] = m_r.addVar(obj=0,vtype=GRB.INTEGER,lb=0,ub=c+2,name='ur'+str(i))
    if i == 0:
        m_r.addConstr(ur[i]==0)
zr = m_r.addVar(vtype=GRB.CONTINUOUS, name = 'zr')

objr = m_r.setObjective(zr, GRB.MINIMIZE)

m_r.addConstr( zr >= gp.quicksum(tau[i,j]*xr[i,j] for i in N0 for j in N1 if j != i))

for v in V:
    m_r.addConstr( zr >= gp.quicksum((tau1[0,i]+tau1[i,c+1]) * yr[i,v] for i in C_S))
    
for j in C:
    m_r.addConstr((gp.quicksum(xr[i,j] for i in N0 if i != j) + gp.quicksum(yr[j,v] for v in V if j in C_S)) == 1)

m_r.addConstr(gp.quicksum(xr[0,j] for j in N1) == 1)

m_r.addConstr(gp.quicksum(xr[i,c+1] for i in N0) == 1)

for j in C:
    m_r.addConstr(gp.quicksum(xr[i,j] for i in N0 if i != j) == gp.quicksum(xr[j,k] for k in N1 if k != j))

m_r.addConstrs(ur[i]-ur[j]+1 <= (c+2)*(1-xr[i,j]) for i in N for j in N1 if j != i);

m_r.optimize()
# Ottimizzazione del modello

LB_r = m_r.ObjVal
LB_r = round(LB_r,3)
print('\n',"LB del problema rilassato: ", LB_r)

## Scelta del miglior LB

In [ ]:
#Prendo tra i due LB calcolati il migliore.
LB=max(LB1,LB_r)
print('\n',"LB: ",LB)
#Il primo metodo, in alcuni dei casi considerati, è risultato migliore di quello del problema rilassato. Più clienti
#appartengono all'insieme C’’, peggiore sarà però tale LB

# Calcolo UB

## UB con TSP del Truck

In [ ]:
#Costruiamo un nuovo modello, uguale a quello precedente ma con la mancanza degli UAV e, dunque, delle variabili y[i,v]
#L'ottimo di questo nuovo modello rappresenta un UB del mio modello di partenza
m_truck = gp.Model('TSP Truck')
xt = m_truck.addVars(tau.keys(), obj=tau, vtype=GRB.BINARY, name = 'xt')
ut = {}
for i in N:
    ut[i] = m_truck.addVar(obj=0,vtype=GRB.INTEGER,lb=0,ub=c+2,name='ut'+str(i))
    if i == 0:
        m_truck.addConstr(ut[i]==0)
zt = m_truck.addVar(vtype=GRB.CONTINUOUS, name = 'zt')

obj_truck = m_truck.setObjective(zt, GRB.MINIMIZE)

m_truck.addConstr( zt >= gp.quicksum(tau[i,j]*xt[i,j] for i in N0 for j in N1 if j != i))
    
for j in C:
    m_truck.addConstr(gp.quicksum(xt[i,j] for i in N0 if i != j) == 1)

m_truck.addConstr(gp.quicksum(xt[0,j] for j in N1) == 1)

m_truck.addConstr(gp.quicksum(xt[i,c+1] for i in N0) == 1)

for j in C:
    m_truck.addConstr(gp.quicksum(xt[i,j] for i in N0 if i != j) == gp.quicksum(xt[j,k] for k in N1 if k != j))

m_truck.addConstrs(ut[i]-ut[j]+1 <= (c+2)*(1-xt[i,j]) for i in N for j in N1 if j != i);


m_truck.optimize()
# Ottimizzazione del modello
m_truck._vars = xt
vals_truck = m_truck.getAttr('X', xt)
tour_truck = subtour(vals_truck)

print('')
print('Tour Ottimo del Truck senza UAV: %s' % str(tour_truck))
UB1 = m_truck.ObjVal
UB1 = round(UB1,3)
print('\n',"UB con TSP Truck: ", UB1)

## UB con metodo pratico

In [ ]:
#Utilizzo quest'altro metodo per il calcolo dell'UB quando non è possibile calcolarlo in tempo utile e veloce con 
#il metodo precedente. Questo avviene quando ho un numero elevato di nodi vicino al deposito.
matrice_vertice_arco_tot = [[tau.get((i,j),0) if i != j else 0 for j in N0] for i in N0] #Costruzione matrice con costi tau 
#Metodo Nearest Neighbor: seleziono iterativamente il nodo successivo che è il più vicino al nodo attuale costrunedo in questo
#modo un percorso che visita tutti i nodi esattamente una volta

def costruisci_ub_nearest_neighbor(matrice_vertice_arco):
    numero_nodi = len(matrice_vertice_arco)
    nodo_attuale = 0  # Iniziamo dal nodo 0
    nodi_non_visitati = set(range(1, numero_nodi))
    percorso = [nodo_attuale]
    costo_totale = 0

    while nodi_non_visitati:
        nodo_successivo = min(nodi_non_visitati, key=lambda x: matrice_vertice_arco[nodo_attuale][x])
        #Trovo il nodo x nell'insieme nodi_non_visitati che ha la distanza minima dal nodo attuale,utilizzando la funzione
        #lambda come criterio di confronto per trovare il nodo più vicino a quello attuale non ancora visitato
        costo_totale += matrice_vertice_arco[nodo_attuale][nodo_successivo]
        percorso.append(nodo_successivo)
        nodi_non_visitati.remove(nodo_successivo)
        nodo_attuale = nodo_successivo

    costo_totale += matrice_vertice_arco[percorso[-1]][percorso[0]]  # Torniamo al nodo di partenza
    return percorso, costo_totale

percorso_nn, ub_nn = costruisci_ub_nearest_neighbor(matrice_vertice_arco_tot)

percorso_nn.append(c+1)
print("Percorso Ammissibile Truck: ", percorso_nn)
print("\nUB_NN: ", ub_nn)
#Tale UB sarà sempre maggiore o al massimo (difficilmente) uguale a quello trovato con il TSP del Truck.

## UB con UAV a servizio unico e TSP del Truck

In [ ]:
# UB con UAV bloccato a servire un numero di clienti pari al numero di UAV
#Costruiamo un nuovo modello, uguale a quello precedente ma con l'insieme C_S (insieme dei nodi raggiungibili dagli UAV) 
#contenente un numero di nodi pari, al più, a quello degli UAV.
#L'ottimo di questo nuovo modello rappresenta un UB del mio modello di partenza

m_UAV = gp.Model('UAV e TSP Truck')
C_S1 = []

t = 0
for i in C_P:
    tau1[0,i]=round(dist[0,i]/vel_UAV,3)
    T_A=tau1[0,i]
    tau1[i,c+1]=round(T_A-T_A*(1-3/4)*P[i-1]/3,3)
    T_R=tau1[i,c+1]
    if t<d:
        if T_A+T_R <= 0.5:
            C_S1.append(i)
            t=t+1

print("Clienti appartenenti all'insieme C'':", '\n', C_S1, '\n')

xu = m_UAV.addVars(tau.keys(), obj=tau, vtype=GRB.BINARY, name = 'xu')
yu = m_UAV.addVars(C_S,d, obj=tau1, vtype=GRB.BINARY,name = 'yu')

uu = {}
for i in N:
    uu[i] = m_UAV.addVar(obj=0,vtype=GRB.INTEGER,lb=0,ub=c+2,name='uu'+str(i))
    if i == 0:
        m_UAV.addConstr(uu[i]==0)

zu = m_UAV.addVar(vtype=GRB.CONTINUOUS, name = 'zu')

obj_UAV = m_UAV.setObjective(zu, GRB.MINIMIZE)

m_UAV.addConstr( zu >= gp.quicksum(tau[i,j]*xu[i,j] for i in N0 for j in N1 if j != i))

for v in V:
    m_UAV.addConstr( zu >= gp.quicksum((tau1[0,i]+tau1[i,c+1]) * yu[i,v] for i in C_S1))
      
for j in C:
    m_UAV.addConstr((gp.quicksum(xu[i,j] for i in N0 if i != j) + gp.quicksum(yu[j,v] for v in V if j in C_S1)) == 1)

m_UAV.addConstr(gp.quicksum(xu[0,j] for j in N1) == 1)

m_UAV.addConstr(gp.quicksum(xu[i,c+1] for i in N0) == 1)

for j in C:
    m_UAV.addConstr(gp.quicksum(xu[i,j] for i in N0 if i != j) == gp.quicksum(xu[j,k] for k in N1 if k != j))

m_UAV.addConstrs(uu[i]-uu[j]+1 <= (c+2)*(1-xu[i,j]) for i in N for j in N1 if j != i);

m_UAV.optimize()
# Ottimizzazione del modello
m_UAV._vars = xu
vals_UAV = m_UAV.getAttr('X', xu)
tour_UAV = subtour(vals_UAV)

print('')
print('Tour Ottimo del Truck senza un numero di nodi pari al numero di UAV: %s' % str(tour_UAV))
UB3 = m_UAV.ObjVal
UB3 = round(UB3,3)
print('\n',"UB con UAV a servizio unico: ", UB3)

## Scelta del miglior UB

In [ ]:
#Prendo tra i tre UB calcolati il migliore.
UB=min(UB1,ub_nn,UB3)
print('\n',"UB: ",UB)